In [22]:
traning_data = [
    ['Green',3,'Mango'],
    ['Yellow',3,'Mango'],
    ['Red',3,'Grape'],
    ['Red',3,'Grape'],
    ['Yellow',3,'Lemon']
]

In [23]:
header = ["color","diameter","label"]

In [24]:
def uniqe_vals(rows,col):
    return set([row[col] for row in rows])

In [25]:
uniqe_vals(traning_data,0)

{'Green', 'Red', 'Yellow'}

In [26]:
uniqe_vals(traning_data,2)

{'Grape', 'Lemon', 'Mango'}

In [51]:
def class_counts(rows):
    counts = {}
    for row in rows:
        label = row[-1]
        if label not in counts:
            counts[label] = 0
            counts[label] += 1
        else:
            counts[label] +=1
    return counts        


In [52]:
class_counts(traning_data)

{'Mango': 2, 'Grape': 2, 'Lemon': 1}

In [53]:
def is_numeric(value):
    return isinstance(value,int) or isinstance(value,float)

In [33]:
is_numeric(7)

True

In [35]:
is_numeric("red")

False

In [36]:
class Question:
    def __init__(self,column,value):
        self.column = column
        self.value = value
        
    def match(self,example):
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value
        
    def __repr__(self):
    condition = "=="
    if is_numeric(self.value):
        condition = ">="
    return "Is %s %s %s?" %(
       header[self.column], condition , str(self.value)
    )        

In [46]:
def partition(rows,Question):
    true_rows,false_rows = [],[]
    for row in rows:
        if Question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows,false_rows   

In [57]:
     # Let's partition the training data based on whether rows are Red.

true_rows, false_rows = partition(traning_data,Question(0,'Red'))

In [58]:
true_rows

[['Red', 3, 'Grape'], ['Red', 3, 'Grape']]

In [59]:
false_rows

[['Green', 3, 'Mango'], ['Yellow', 3, 'Mango'], ['Yellow', 3, 'Lemon']]

In [54]:
def gini(rows):
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity    

In [63]:
gini(traning_data)

0.6399999999999999

In [64]:
def info_gain(left,right,current_uncertainty):
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p*gini(left) - (1-p)*gini(right)

In [65]:
current_uncertainty = gini(traning_data)
true_rows, false_row = partition(traning_data,Question(0,'Red'))
info_gain(true_rows,false_rows,current_uncertainty)

0.37333333333333324

In [66]:
current_uncertainty = gini(traning_data)
true_rows, false_row = partition(traning_data,Question(0,'Green'))
info_gain(true_rows,false_rows,current_uncertainty)

0.30666666666666653

In [67]:
current_uncertainty = gini(traning_data)
true_rows, false_row = partition(traning_data,Question(0,'Yellow'))
info_gain(true_rows,false_rows,current_uncertainty)

0.17333333333333323

In [74]:
def find_best_split(rows):
    best_gain = 0
    best_question = None
    current_uncertainty = gini(rows)
    n_features = len(rows[0])-1
    
    for col in range(n_features):
        values = set([row[col] for row in rows])
        
        for val in values:
            
            question = Question(col,val)
            
            true_rows,false_rows = partition(rows,question)
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue
                
            gain = info_gain(true_rows,false_rows,current_uncertainty)
            
            if gain >= best_gain:
                best_gain, best_question = gain , question
                
    return best_gain,best_question            

In [75]:
best_gain,best_question = find_best_split(traning_data)

In [76]:
best_gain

0.37333333333333324

In [78]:
best_question 

In [96]:
class Leaf:
    def __init__(self,rows):
        self.predictions = class_counts(rows)

In [97]:
class Decision_Node:
    def __init__(self,question,true_branch,false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [98]:
def build_tree(rows):
    gain,question = find_best_split(rows)
    
    if gain == 0:
        return Leaf(rows)
    
    true_rows,false_rows = partition(rows,question)
    
    true_branch = build_tree(true_rows)
    false_brance = build_tree(false_rows)
    
    return Decision_Node(question,true_branch,false_branch)

In [99]:
def print_tree(node,spacing=""):
    if isinstance(node,Leaf):
        print(spacing + "predict",node.predictions)
        return
    print(spacing + str(node.question))
    print(spacing + '--> True:')
    print_tree(node.true_branch,spacing + "  ")
    print_tree(node.false_branch,spacing + "  ")

In [100]:
def classify(row,node):
    if isinstance(node,Leaf):
        return node.predictions
    
    if node.question.match(row):
        return classify(row,node.true_branch)
    else:
         return classify(row,node.false_branch)
        

In [ ]:
my_tree = build_tree(traning_data)
classify(traning_data[0],my_tree)

In [102]:
def print_leaf(counts):
    total = sum(count.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(count[lbl]/total*100)) + "%"
        
    return probs    

In [ ]:
print_leaf(classify(traning_data[0],my_tree))

In [ ]:
if __name__ == '__main__':
    my_tree = build_tree(traning_data)
    print_tree(my_tree)
    
    testing_data = [
    ['Green',3,'Apple'],
    ['Yellow',4,'Apple'],
    ['Red',2,'Grape'],
    ['Red',1,'Grape'],
    ['Yellow',3,'Lemon']
]
    
    for row in testing_data:
        print("Actual:%s. predicted: %s" %(rows[-1],print_leaf(classify(row,my_tree))))